In [1]:
import pandas as pd
from datetime import datetime
import pyspark.sql.functions as functions
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from functools import *
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, expr, when
import math
from geopy import distance
import networkx as nx
import datetime
from datetime import timezone
import math
import numpy as np

In [ ]:
start_date = datetime.date(2017, 9, 13)
end_date = datetime.date(2017, 9, 30)

In [2]:
path = "2017-09/*.csv"
station_path = "metadata/BFKOORD_GEO"
BPUIC = []
long = []
lat = []
name = []
zurich_coord = (47.407665036,8.540331172)
MAX_DIST = 10

In [3]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-mbanga>

In [4]:
spark = SparkSession(sc)

In [5]:
#r = pd.read_pickle('september')

In [6]:
path = "/home/mbanga/BFKOORD_GEO"
station_file = open(path,'r')


for line in station_file:
    l = line.split()
    BPUIC.append(l[0])
    long.append(l[1])
    lat.append(l[2])
    name.append(l[5])
    
d = {'BPUIC':BPUIC,
     'longitude' : long,
     'latitude' : lat,}

station_df = pd.DataFrame(d)
station_df = station_df[['BPUIC','longitude','latitude']]
station_df['BPUIC'] = station_df['BPUIC'].astype(np.int64)

In [7]:
station_df.head()

,BPUIC,longitude,latitude
0,2,26.074412,44.446770
1,3,1.811446,50.901549
2,4,1.075329,51.284212
3,5,-3.543547,50.729172
4,7,9.733756,46.922368


In [8]:
df_station = spark.createDataFrame(station_df)


In [9]:
df_station.show()

+-----+---------+---------+
|BPUIC|longitude| latitude|
+-----+---------+---------+
|    2|26.074412|44.446770|
|    3| 1.811446|50.901549|
|    4| 1.075329|51.284212|
|    5|-3.543547|50.729172|
|    7| 9.733756|46.922368|
|    8| 8.571251|50.051219|
|    9|18.643803|54.355520|
|   11| 7.389462|47.191804|
|   13|29.019602|40.996348|
|   17| 9.873959|48.577852|
|   19| 4.786044|43.921937|
|   21| 2.140369|41.378914|
|   22| 7.589551|47.547405|
|   24| 7.395229|46.937482|
|   27|-1.899480|52.483627|
|   28| 6.838953|46.949588|
|   29|17.106466|48.158910|
|   31| 4.335694|50.835376|
|   34|-2.979650|53.404289|
|   35| 8.500049|47.114619|
+-----+---------+---------+
only showing top 20 rows



In [10]:
def clean_distance(latitude,longitude):
    coord = (latitude,longitude)
    print(latitude)
    if(latitude == "null" or longitude == "null"):
        return False
    else:
        distance_ = distance.distance(zurich_coord,coord).km
        return distance_ < MAX_DIST

In [11]:
from datetime import timedelta, date
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [13]:
start_hdfs_file_path = 'hdfs://iccluster045.iccluster.epfl.ch:8020/datasets/project/istdaten/'
end_hdfs_file_path = 'istdaten.csv'
slash = '/'

r = pd.DataFrame()
for single_date in daterange(start_date, end_date):
        read_path = start_hdfs_file_path + str(single_date.year) + slash + str(single_date.month).zfill(2) + slash
        read_path += single_date.strftime("%Y-%m-%d") + end_hdfs_file_path

        df = spark.read.csv(read_path, sep=';',header=True) 
        df = df.select(col("BETRIEBSTAG").alias("trip_date"), col("FAHRT_BEZEICHNER").alias("trip_id")
                  , col("BETREIBER_ID").alias("operator_id"), col("BETREIBER_ABK").alias("operator_abk")
                  , col("BETREIBER_NAME").alias("operator_name"), col("PRODUKT_ID").alias("product_id"), col("LINIEN_ID").alias("line_id")
                  , col("LINIEN_TEXT").alias("line_text"), col("UMLAUF_ID").alias("circulation_id"), col("VERKEHRSMITTEL_TEXT").alias("transport_text")
                  , col("ZUSATZFAHRT_TF").alias("additional_trip"), col("FAELLT_AUS_TF").alias("failed"), col("BPUIC").alias("BPUIC")
                  , col("HALTESTELLEN_NAME").alias("stop_name"), col("ANKUNFTSZEIT").alias("scheduled_arrival_time"), col("AN_PROGNOSE").alias("actual_arrival_time")
                  , col("AN_PROGNOSE_STATUS").alias("actual_arrival_time_status"), col("ABFAHRTSZEIT").alias("scheduled_departure_time"), col("AB_PROGNOSE").alias("actual_departure_time")
                  , col("AB_PROGNOSE_STATUS").alias("actual_departure_time_status"), col("DURCHFAHRT_TF").alias("no_stop"))

        df = df.join(df_station,['BPUIC'],"outer")

        filter_udf = udf(clean_distance)
        df = df.withColumn("distance",filter_udf(df.latitude,df.longitude))
        g = df.filter(df.distance==True)
        
        r = g.toPandas()
        r.to_pickle('original_dfs/'+str(single_date))
        #r.to_csv(save_path, compression='gzip')
        #to_concat = g.toPandas()
        #r = pd.concat([r, to_concat], axis=0)      

In [15]:
import sys
#print(sys.getsizeof(r) / float(10**6), 'MB')